In [1]:
from mimicfouretl.data_insights import create_ui

In [2]:
create_ui()

Dropdown(description='Dataset:', options=('hosp/hospital_admissions (copy)', 'hosp/hospital_admissions'), valu…

Output()